In [2]:
from dataset_collection import dataset
from utils import get_dataset, get_model
import cv2
font = cv2.FONT_HERSHEY_DUPLEX
import numpy as np 
import keras
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils.np_utils import to_categorical   

In [21]:
## Getting the dataset

live_img, live_img_label, not_live, not_live_label = get_dataset()

img = live_img + not_live
labels = live_img_label+ not_live_label
images=[]
for i in img:
    img = cv2.imread(i)
    img = cv2.resize(img, (100,100))
    images.append(img)

X = np.array(images, dtype=float)
y = np.array(labels, dtype=float)
y= y.reshape((-1,1))
X /= 255
y = to_categorical(y, num_classes=2)

In [22]:
X.shape, y.shape

((4000, 100, 100, 3), (4000, 2))

In [23]:
## Model Training

model = get_model()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit(x=X, y=y, validation_split=0.1, epochs=3)

Epoch 1/3
113/113 [==============================] - 95s 841ms/step - loss: 0.0675 - acc: 0.9661 - val_loss: 7.9171e-06 - val_acc: 1.0000
Epoch 2/3
113/113 [==============================] - 92s 813ms/step - loss: 9.8137e-05 - acc: 1.0000 - val_loss: 1.4421e-06 - val_acc: 1.0000
Epoch 3/3
113/113 [==============================] - 88s 780ms/step - loss: 1.6460e-04 - acc: 1.0000 - val_loss: 3.3110e-07 - val_acc: 1.0000


In [30]:
## Save weights
#model.save_weights('weights.h5')
model = get_model()
model.load_weights('weights.h5')

In [31]:
########## KNN CODE ############
def distance(v1, v2):
    # Eucledian 
    return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
    dist = []

    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]

    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]
################################

In [32]:
dataset_path = "face_dataset/"

face_data = []
labels = []
class_id = 0
names = {}

In [33]:
import os

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)

        target = class_id * np.ones((data_item.shape[0],))    ##names= {[0,vasu],[1,vinayak],....so on}
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1, 1))
print(face_labels.shape)
print(face_dataset.shape)

trainset = np.concatenate((face_dataset, face_labels), axis=1)
print(trainset.shape)

(58, 1)
(58, 30000)
(58, 30001)


In [42]:
cap = cv2.VideoCapture(0)

#result = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, (640, 480))

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    img = cv2.resize(frame, (100,100))
    img = np.expand_dims(img, axis=0)
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    pred = model.predict(img)
    
    if pred[0][1]:
        cv2.putText(frame, '!!WARNING!!', (frame.shape[1]//2, frame.shape[0]//2), font, 1.0, (255, 255, 255), 1)        
    
    else:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:

            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            
            offset = 5
            face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
            face_section = cv2.resize(face_section, (100, 100))
            
            out = knn(trainset, face_section.flatten())
            cv2.putText(frame, names[int(out)],(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),2,cv2.LINE_AA)
        
        cv2.putText(frame, 'UNLOCKED', (frame.shape[1]//2, frame.shape[0]//2), font, 1.0, (0, 255, 0), 1)

    #result.write(frame)

    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
#result.release()
cv2.destroyAllWindows()